### 1. Install the libraries

In [ ]:

#Install prerequisite packages
!pip install python-dotenv==1.0.0
!pip install llama-index
!pip install openai
!pip install "transformers[torch]"
!pip install huggingface_hub[hf_xet]
!pip install -U bitsandbytes

  Using cached python_dotenv-1.0.0-py3-none-any.whl.metadata (21 kB)
Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)
  Attempting uninstall: python-dotenv
    Found existing installation: python-dotenv 1.1.0
    Uninstalling python-dotenv-1.1.0:
      Successfully uninstalled python-dotenv-1.1.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-cloud-services 0.6.20 requires python-dotenv<2.0.0,>=1.0.1, but you have python-dotenv 1.0.0 which is incompatible.


  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
Using cached python_dotenv-1.1.0-py3-none-any.whl (20 kB)
  Attempting uninstall: python-dotenv
    Found existing installation: python-dotenv 1.0.0
    Uninstalling python-dotenv-1.0.0:
      Successfully uninstalled python-dotenv-1.0.0


### Step 1: Patch asyncio & set your OpenAI key

In [ ]:

import os
import nest_asyncio

# 1) Set your OpenAI key and patch asyncio for notebooks
os.environ["OPENAI_API_KEY"] = "sk...."  # ← your OpenAI secret key here
nest_asyncio.apply()

### Step 2: Import core libraries

In [3]:
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import QueryEngineTool
from llama_index.core.selectors import LLMSingleSelector
from llama_index.core.query_engine import RouterQueryEngine

### Step 3: Configure embeddings & RAG completion model

In [4]:
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")
retriever_llm = OpenAI(
    model="gpt-3.5-turbo-16k",
    temperature=0.5,
    max_tokens=256,
)

### Step 4: Load & index your PDFs

In [5]:
splitter = SentenceSplitter(chunk_size=1024)

# HR handbook
hr_docs  = SimpleDirectoryReader(input_files=["HR_Policy_Handbook.pdf"]).load_data()
hr_nodes = splitter.get_nodes_from_documents(hr_docs)
hr_index = VectorStoreIndex(hr_nodes)
hr_qe    = hr_index.as_query_engine(llm=retriever_llm)

# IT security guidelines
it_docs  = SimpleDirectoryReader(input_files=["IT_Security_Guidelines.pdf"]).load_data()
it_nodes = splitter.get_nodes_from_documents(it_docs)
it_index = VectorStoreIndex(it_nodes)
it_qe    = it_index.as_query_engine(llm=retriever_llm)

### Step 5: Wrap engines as tools & build the router

In [6]:
# Wrap as tools with name+description
hr_tool = QueryEngineTool.from_defaults(
    query_engine=hr_qe,
    name="HR_Policy_Agent",
    description="Handles questions about HR policies: leave, benefits, onboarding."
)
it_tool = QueryEngineTool.from_defaults(
    query_engine=it_qe,
    name="IT_Security_Agent",
    description="Handles questions about IT security: passwords, encryption, access control."
)

# Deterministic router LLM
router_llm = OpenAI(
    model="gpt-3.5-turbo",
    temperature=0.0,
    max_tokens=64,
)
# Build selector + router
selector = LLMSingleSelector.from_defaults(llm=router_llm)
router   = RouterQueryEngine(
    selector=selector,
    query_engine_tools=[hr_tool, it_tool],
)

### Step 6: Ask questions with one call

In [7]:
print("HR →", router.query("What is our leave policy if I join mid-year?"))
print("IT →", router.query("How often must I rotate my password?"))

HR → You would be entitled to a prorated amount of the 20 paid vacation days and 10 sick leave days based on the time remaining in the year when you join.
IT → Change every 90 days; never reuse old passwords.
